In [1]:
import ezdxf

fname = "../dxffiles/P-E-13,5-BS-Master-09.dxf"
d = ezdxf.readfile(fname)
print("layerspresent:", set(e.dxf.layer for e in d.entities))


layerspresent: {'BSF-CUT-BOTH', 'BSR-CUT-EVO', 'BSM-CUT-BOTH', 'BSM-PLOT-EVO', 'BSR-PLOT-PUMA', 'PATCH-OUTSIDE', 'PATCH-MYLAR', 'BSR-CUT-PUMA', 'CONSTRUCTION', 'BSF-PLOT-PUMA', 'BSM-PLOT-PUMA', 'SPLINES-MASTER', 'PATCH-INSIDE', 'BSF-PLOT-EVO', 'PATCH-REF', 'BSR-PLOT-EVO', 'PUMA-REF', 'HISTORY'}


In [2]:
keeplayers = ["BSF-CUT-BOTH", 'BSF-PLOT-PUMA']
selectedname = "BSF-PUMA"

doc = ezdxf.new('R12')
block = doc.blocks.new(name=selectedname)
doc.layers.new("1")


<class 'ezdxf.entities.layer.Layer'> LAYER(#30)

In [3]:
for e in d.entities:
    dxfattribs = {"layer":"1"}
    if e.dxf.layer in keeplayers:
        if e.dxftype() == "LINE":
            block.add_line(e.dxf.start, e.dxf.end, dxfattribs=dxfattribs)
        elif e.dxftype() == "ARC":
            block.add_arc(e.dxf.center, e.dxf.radius, e.dxf.start_angle, e.dxf.end_angle, dxfattribs=dxfattribs)
        elif e.dxftype() == "SPLINE":
            import nurbs.Curve
            curve = nurbs.Curve.Curve()
            curve.ctrlpts = [p[:2]  for p in e.control_points]
            curve.degree = e.dxf.degree
            curve.knotvector = e.knots
            #curve.knotvector = nurbs.utilities.knotvector_autogen(curve.degree, len(curve.ctrlpts))
            #curve.weights = e.weights
            curve.delta = 0.01
            curve.evaluate_rational()
            block.add_polyline2d(curve.curvepts, dxfattribs=dxfattribs)
        else:
            print(e.dxftype())
            block.add_foreign_entity(e)


ModuleNotFoundError: No module named 'nurbs'

In [86]:
msp = doc.modelspace()
k = msp.add_blockref(selectedname, (0, 0, 0), dxfattribs={'rotation': 0, "layer":"1"})
doc.saveas("test.dxf")